In [ ]:
import os
import random
import numpy as np
import tensorflow as tf

# Fix randomness เพื่อ reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# 1. Load data
df = yf.download('GC=F', start='2020-01-01', end='2025-07-20')
df = df[['Close']].dropna()

# 2. Scale
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(df)

# 3. Sequence
SEQ_LEN = 30
def make_sequences(data, seq_len=30):
    X, y = [], []
    for i in range(seq_len, len(data)):
        X.append(data[i-seq_len:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)
X, y = make_sequences(scaled, SEQ_LEN)

# 4. Split train/test
split = int(len(X)*0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
X_train = X_train.reshape((-1, SEQ_LEN, 1))
X_test = X_test.reshape((-1, SEQ_LEN, 1))

# 5. Model definition
def build_lstm():
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(SEQ_LEN, 1)),
        Dropout(0.2),
        LSTM(32),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# 6. Train N รอบ เลือก best in-memory (ไม่ save/load ไฟล์)
# สมมติว่า Accuracy หมายถึง "คาดการณ์ผิดไม่เกิน 2% ของราคาจริง"
def regression_accuracy(y_true, y_pred, tolerance=0.05):
    return np.mean(np.abs(y_true - y_pred) / y_true <= tolerance)

n_runs = 5
results = []
best_rmse = float('inf')
best_model = None
best_weights = None
best_r2 = None

SEED = 42  # ตั้ง seed คงที่

for i in range(n_runs):
    # Set seed for reproducibility
    import random
    random.seed(SEED + i)
    np.random.seed(SEED + i)
    tf.random.set_seed(SEED + i)

    model = build_lstm()
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
    ]
    history = model.fit(
        X_train, y_train, epochs=50, batch_size=32,
        validation_data=(X_test, y_test), verbose=0, callbacks=callbacks
    )
    y_pred = model.predict(X_test)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred_inv = scaler.inverse_transform(y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    mae = mean_absolute_error(y_test_inv, y_pred_inv)
    r2 = r2_score(y_test_inv, y_pred_inv)
    acc = regression_accuracy(y_test_inv.flatten(), y_pred_inv.flatten(), tolerance=0.05)  # 2% tolerance
    results.append({'Run': i+1, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'Accuracy': acc})
    if rmse < best_rmse:
        best_rmse = rmse
        best_r2 = r2
        best_model = model
        best_weights = model.get_weights()


# 7. สรุปผลแต่ละรอบ
results_df = pd.DataFrame(results)
print("\n=== Summary of all LSTM runs ===")
print(results_df)

# 8. Restore best weights
if best_model is not None and best_weights is not None:
    best_model.set_weights(best_weights)
    y_pred = best_model.predict(X_test)
    y_pred_inv = scaler.inverse_transform(y_pred)
    y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
    # กราฟเปรียบเทียบ
    plt.figure(figsize=(12, 5))
    plt.plot(df.index[-len(y_test_inv):], y_test_inv, label='Actual')
    plt.plot(df.index[-len(y_test_inv):], y_pred_inv, label='Best LSTM Predicted')
    plt.legend()
    plt.title(f'Best LSTM Forecast vs Actual (R2={best_r2:.3f})')
    plt.show()
    print(f"\nBest RMSE={best_rmse:.2f}, R2={best_r2:.3f}")

# 9. ทำนายอนาคต N วัน (ใช้ best_model)
future_days = 120
last_sequence = scaled[-SEQ_LEN:].reshape(1, SEQ_LEN, 1)
future_preds = []
for _ in range(future_days):
    next_pred = best_model.predict(last_sequence)[0][0]
    future_preds.append(next_pred)
    last_sequence = np.append(last_sequence[:, 1:, :], [[[next_pred]]], axis=1)
future_preds_inv = scaler.inverse_transform(np.array(future_preds).reshape(-1, 1))
future_index = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=future_days, freq='B')

plt.figure(figsize=(14, 6))
# เส้น Actual (ทุกจุด)
plt.plot(df.index, df['Close'], label='Actual', color='blue', linewidth=1.5)
# เส้น Predicted (เฉพาะช่วง test)
plt.plot(df.index[-len(y_test_inv):], y_pred_inv, label='LSTM Predicted (Test)', color='orange', linewidth=2)
# เส้น Forecast อนาคต
plt.plot(future_index, future_preds_inv, label='Forecast 30 Days Ahead', color='red', linestyle='dashed', linewidth=2)
plt.legend()
plt.title('LSTM Gold Price Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('Price')
plt.tight_layout()
plt.show()

# Optional: print out future forecast
future_df = pd.DataFrame({'Forecast': future_preds_inv.flatten()}, index=future_index)
print(future_df.head(30))


In [ ]:
# 10. Evaluate model performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# หลังได้ y_test_inv (จริง) กับ y_pred_inv (โมเดลทำนาย)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
mape = np.mean(np.abs((y_test_inv - y_pred_inv) / y_test_inv)) * 100
r2 = r2_score(y_test_inv, y_pred_inv)

# กำหนด Accuracy แบบ custom: ทายใกล้จริงในช่วง ±3%
accuracy = np.mean(np.abs((y_pred_inv - y_test_inv) / y_test_inv) <= 0.03)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"R²: {r2:.3f}")
print(f"Accuracy (±3%): {accuracy*100:.2f}%")

# Learning curve (loss/val_loss)
plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curve')
plt.legend()
plt.show()


In [ ]:
# Plot residuals
residuals = (y_test_inv.flatten() - y_pred_inv.flatten())

plt.figure(figsize=(12,4))
plt.plot(df.index[-len(residuals):], residuals, marker='o', linestyle='-', color='purple')
plt.axhline(0, color='gray', linestyle='--')
plt.title("Residuals vs Time")
plt.xlabel("Date")
plt.ylabel("Residual (Actual - Predicted)")
plt.show()



In [ ]:
# Plot histogram of residuals
plt.figure(figsize=(7,4))
plt.hist(residuals, bins=30, color='teal', alpha=0.8, edgecolor='black')
plt.title("Histogram of Residuals")
plt.xlabel("Residual (Actual - Predicted)")
plt.ylabel("Frequency")
plt.show()


In [ ]:
# Plot predicted vs actual
plt.figure(figsize=(6,6))
plt.scatter(y_test_inv, y_pred_inv, alpha=0.6)
plt.plot([y_test_inv.min(), y_test_inv.max()],
         [y_test_inv.min(), y_test_inv.max()],
         color='red', linestyle='--', label='Perfect Prediction')
plt.title("Predicted vs Actual")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.legend()
plt.show()
